In [1]:
#pip install fastf1

In [2]:
import fastf1
from fastf1 import plotting
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from math import pi
from fastf1.ergast import Ergast
#ergast = Ergast()
# Enable the cache
fastf1.Cache.enable_cache(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\F1 Research\f1cache")

import warnings
warnings.filterwarnings("ignore")

In [3]:
def time_to_millis(time_str):
    if pd.isna(time_str):
        return None
    try:
        minutes, seconds = time_str.split(':')
        seconds, millis = seconds.split('.')
        total_millis = int(minutes) * 60000 + int(seconds) * 1000 + int(millis)
        return total_millis
    except ValueError:
        return None  # or handle the error as you see fit


In [4]:
def calculate_advance(row):
    q1, q2, q3 = row['q1'], row['q2'], row['q3']
    if pd.notna(q1) and pd.notna(q2) and pd.notna(q3):
        return 3
    elif pd.notna(q1) and pd.notna(q2) and pd.isna(q3):
        return 2
    elif pd.notna(q1) and pd.isna(q2) and pd.isna(q3):
        return 1
    else:
        return 0


In [5]:
qdf = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\qualifying.csv")
rdf = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\results.csv")
df = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\results.csv")
sdf = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\sprint_results.csv")
ddf = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\drivers.csv")
cdf = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\constructors.csv")
radf = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\races.csv")
laps = pd.read_csv(r"C:\Users\BenjaminNoe\OneDrive - NCD Agency LLC (NCRDNTL)\Desktop\Sports Analytics\F1 Visualization\archive\lap_times.csv")

In [6]:
average_times = laps.groupby(['raceId', 'driverId'])['milliseconds'].mean().reset_index()
average_times.rename(columns={'milliseconds': 'average_milliseconds'}, inplace=True)
average_times.head()

,raceId,driverId,average_milliseconds
0,1,1,97563.758621
1,1,2,97635.672414
2,1,3,97612.172414
3,1,4,97597.637931
4,1,6,91822.235294


In [7]:
qdf.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236


In [8]:
radf.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [9]:
#this is how to join two dataframes, we have a new column from radf in qdf now

qdf = pd.merge(qdf, radf[['raceId', 'year']], on='raceId', how='left')
qdf['q1'] = qdf['q1'].apply(time_to_millis)
qdf['q2'] = qdf['q2'].apply(time_to_millis)
qdf['q3'] = qdf['q3'].apply(time_to_millis)
qdf['advance'] = qdf.apply(calculate_advance, axis=1)
qdf = qdf[qdf['advance'] != 0] #remove all drivers who didnt record a time

qdf.head()

#qdf.to_csv('qdfTEST.csv')

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,year,advance
0,1,18,1,1,22,1,86572.0,85187.0,86714.0,2008,3
1,2,18,9,2,4,2,86103.0,85315.0,86869.0,2008,3
2,3,18,5,1,23,3,85664.0,85452.0,87079.0,2008,3
3,4,18,13,6,2,4,85994.0,85691.0,87178.0,2008,3
4,5,18,2,2,3,5,85960.0,85518.0,87236.0,2008,3


In [10]:
qdf_copy = qdf[qdf['advance'] == 3]
qdf_copy = qdf_copy.groupby('raceId')[['q1', 'q2', 'q3']].mean().reset_index()
qdf_copy.rename(columns={'q1': 'avg_q1', 'q2': 'avg_q2', 'q3': 'avg_q3'}, inplace=True)
# Merge the average times back into the original DataFrame
qdf = qdf.merge(qdf_copy[['raceId', 'avg_q1','avg_q2','avg_q3']], on='raceId', how='left')
qdf.head()
qdf.to_csv('qdf_test2.csv')

In [11]:
rdf = pd.merge(rdf, radf[['raceId', 'year']], on='raceId', how='left')
df = pd.merge(df, radf[['raceId', 'year', 'round']], on='raceId', how='left')


rdf = pd.merge(rdf, average_times[['raceId', 'driverId','average_milliseconds']], on=['raceId', 'driverId'], how='left') #average lap time for constructor strength

rdf.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,average_milliseconds
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,2008,98114.068966
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,2008,98208.517241
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,2008,98254.810345
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,2008,98410.293103
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,2008,98424.655172


In [12]:
def replace_outliers_with_nan(df, session):
    Q1 = df[session].quantile(0.25)
    Q3 = df[session].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Replace outliers with NaN
    df[session] = df[session].apply(lambda x: np.nan if x < lower_bound or x > upper_bound else x)
    return df

def process_columns(df, columns):
    for column in columns:
        df = replace_outliers_with_nan(df, column)
    return df

In [13]:
#INCLUSIVE
#1994-2004 ONLY Q1
#2005 MOSTLY Q1 WITH SOME Q2
#2006-PRESENT Q1, Q2, Q3
qdf['average_q'] = np.nan
for i in range(0, len(qdf)):
    if qdf['year'][i] < 2005:
        qdf['average_q'][i] = qdf['q1'][i]
    elif qdf['year'][i] == 2005:
        if qdf['advance'][i] == 2:      
            qdf['average_q'][i] = (qdf['q1'][i] + qdf['q2'][i])/2
        else:
            qdf['average_q'][i] = qdf['q1'][i]
    else:
        #for every race, 
        if(pd.isna(qdf['q2'][i]) and pd.isna(qdf['q3'][i])): #out in q1
            #q1diff = qdf['q1'][i] / qdf['avg_q1'][i]
            #first q2
            versus1_2 = qdf['avg_q2'][i] / qdf['avg_q1'][i]
            versus1_3 = qdf['avg_q3'][i] / qdf['avg_q1'][i]
            
            qdf['q2'][i] = qdf['q1'][i] * versus1_2
            qdf['q3'][i] = qdf['q1'][i] * versus1_3
            
        elif(pd.isna(qdf['q3'][i])): #out in q2
            versus1_3 = qdf['avg_q3'][i] / qdf['avg_q1'][i]
            qdf['q3'][i] = qdf['q1'][i] * versus1_3
        else:
            pass
            
        qdf['average_q'][i] = (qdf['q1'][i] + qdf['q2'][i] + qdf['q3'][i])/3
            
        

In [14]:
qdf.to_csv('qdf_test3.csv')

In [31]:
subset = qdf[qdf['year'] >= 2006]
rest = qdf[qdf['year'] < 2006]

subset = subset.groupby('raceId').apply(lambda x: process_columns(x, ['q1', 'q2', 'q3'])).reset_index(drop=True)
qdf = pd.concat([subset, rest], ignore_index=True)
qdf['average_q'] = qdf[['q1', 'q2', 'q3']].mean(axis=1, skipna=True)
qdf

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,year,advance,avg_q1,avg_q2,avg_q3,average_q
0,2986,1,18,23,22,1,85211.0,84855.0,86202.0,2009,3,85678.6,85152.0,86896.8,85422.666667
1,2987,1,22,23,23,2,85006.0,84783.0,86505.0,2009,3,85678.6,85152.0,86896.8,85431.333333
2,2988,1,20,9,15,3,85938.0,85121.0,86830.0,2009,3,85678.6,85152.0,86896.8,85963.000000
3,2989,1,9,2,5,4,85922.0,85152.0,86914.0,2009,3,85678.6,85152.0,86896.8,85996.000000
4,2990,1,3,3,16,5,85846.0,85123.0,86973.0,2009,3,85678.6,85152.0,86896.8,85980.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10095,4532,76,27,18,21,14,79229.0,82977.0,NaN,2005,2,NaN,NaN,NaN,81103.000000
10096,4533,76,33,17,18,15,79408.0,83670.0,NaN,2005,2,NaN,NaN,NaN,81539.000000
10097,4534,76,31,1,10,16,74858.0,NaN,NaN,2005,1,NaN,NaN,NaN,74858.000000
10098,4535,76,39,17,19,17,79474.0,83948.0,NaN,2005,2,NaN,NaN,NaN,81711.000000


In [33]:
qdf.to_csv('qdf_test4.csv')

In [35]:
driver = pd.merge(rdf[['raceId', 'driverId', 'constructorId', 'average_milliseconds', 'year']], qdf[['raceId', 'driverId','average_q']], on=['raceId', 'driverId'], how='left') #average lap time for constructor strength
grouped = driver.groupby(['raceId', 'constructorId']).agg({
    'average_milliseconds': 'mean',
    'average_q': 'max',
    'year': 'first'  # or 'max', 'min', depending on the appropriate logic for your context
}).reset_index()

# Display the grouped DataFrame
#grouped = grouped[grouped['raceId'] != 1132]
grouped['constructor_average'] = grouped[['average_milliseconds', 'average_q']].mean(axis=1)

grouped


,raceId,constructorId,average_milliseconds,average_q,year,constructor_average
0,1,1,97563.758621,86686.619683,2009,92125.189152
1,1,2,96398.181661,86126.103330,2009,91262.142496
2,1,3,94717.203854,86326.273966,2009,90521.738910
3,1,4,99681.610632,86831.007141,2009,93256.308886
4,1,5,97619.568966,87197.991928,2009,92408.780447
...,...,...,...,...,...,...
12903,1132,117,NaN,88311.000000,2024,88311.000000
12904,1132,131,NaN,87549.333333,2024,87549.333333
12905,1132,210,NaN,88371.333333,2024,88371.333333
12906,1132,214,NaN,NaN,2024,NaN


In [39]:
driver.to_csv('qdf_test4.5.csv')

In [16]:
min_times = grouped.groupby('raceId')['constructor_average'].min().rename('min_total_average')
grouped = grouped.merge(min_times, on='raceId')
grouped['percent_difference'] = grouped['min_total_average'] / grouped['constructor_average']

grouped

,raceId,constructorId,average_milliseconds,average_q,year,constructor_average,min_total_average,percent_difference
0,1,1,97563.758621,86686.619683,2009,92125.189152,90521.738910,0.982595
1,1,2,96398.181661,86126.103330,2009,91262.142496,90521.738910,0.991887
2,1,3,94717.203854,86326.273966,2009,90521.738910,90521.738910,1.000000
3,1,4,99681.610632,86831.007141,2009,93256.308886,90521.738910,0.970677
4,1,5,97619.568966,87197.991928,2009,92408.780447,90521.738910,0.979579
...,...,...,...,...,...,...,...,...
12903,1132,117,NaN,88311.000000,2024,88311.000000,87549.333333,0.991375
12904,1132,131,NaN,87549.333333,2024,87549.333333,87549.333333,1.000000
12905,1132,210,NaN,89895.174591,2024,89895.174591,87549.333333,0.973905
12906,1132,214,NaN,NaN,2024,NaN,87549.333333,NaN


In [17]:
df = pd.merge(df, grouped[['percent_difference', 'constructor_average','min_total_average','average_q','raceId', 'constructorId']], on=['raceId', 'constructorId'], how='left')
df = df[df['year'] >= 1994] 
df.to_csv('datatest.csv')

### ELO START (NOW OUTDATED)

In [19]:
ddf['ELO'] = 1200
ddf['MAX ELO'] = 1200

In [20]:
#ELO TEST LOOP
df = pd.merge(df, grouped[['percent_difference', 'raceId', 'constructorId']], on=['raceId', 'constructorId'], how='left')
df = df[df['raceId'] != 1132]
df['previous_elo'] = np.nan
df['updated_elo'] = np.nan
df['driverId'] = df['driverId'].astype(int)
ddf['driverId'] = ddf['driverId'].astype(int)  # Ensure both columns are of the same type



df['position'] = df['position'].replace('\\N', '1000')

# Convert the column to an integer type
df['position'] = df['position'].astype(int)
df['position'] = df['position'].fillna(1000)

#df = df[df['year'] > 1993]

#df = df.dropna(subset=['percent_difference'])

# Verify that rows have been dropped
print(df['percent_difference'].isnull().sum()) 

df = df.sort_values(by=['year', 'round'], ascending=[True, True])

df.reset_index(drop=True, inplace=True)

df

KeyError: 'percent_difference'

In [ ]:
races = df['raceId'].unique()

In [ ]:
def calculate_elo_change(row, opp_row):
    k_factor = 1
    


    # Adjust the base ELO difference based on performance percentage and finish status
    base_elo_row = row['previous_elo'] / (row['percent_difference'] ** 15) #if row['position'] != 1000 else 1)  # Reduce influence if not finished
    base_elo_opp = opp_row['previous_elo'] / (opp_row['percent_difference'] ** 15) #if opp_row['position'] != 1000 else 1)

    # Calculate the ELO difference
    adjusted_elo_diff = base_elo_row - base_elo_opp
    try:
        expected_score = 1 / (1 + 10 ** (adjusted_elo_diff / 400))
    except OverflowError:
        # If adjusted_elo_diff is extremely positive, expected_score approaches 0
        # If adjusted_elo_diff is extremely negative, expected_score approaches 1
        expected_score = 0 if adjusted_elo_diff > 0 else 1

    # Determine the actual score based on positions, handling non-finishes
    if row['position'] == 1000:
        actual_score = 0
    elif opp_row['position'] == 1000:
        actual_score = 1  # Win against a non-finisher
    else:
        actual_score = 1 if row['position'] < opp_row['position'] else 0

    # Calculate ELO change
    elo_change = k_factor * (actual_score - expected_score)

        
    if row['position'] == 1000 and opp_row['position'] == 1000:
        elo_change = 0
    
    #if row['driverId'] == 30 or opp_row['driverId'] == 30:
        #print(f"Matchup: {row['forename']} {row['position']} vs {opp_row['forename']} {opp_row['position']}, ELO Change: {elo_change}")    
    
    return elo_change


In [ ]:
for race_id in races:
    # Create a subset for the current race
    current_race = df[df['raceId'] == race_id].copy()

    # Drop 'previous_elo' if it already exists in current_race
    if 'previous_elo' in current_race.columns:
        current_race.drop('previous_elo', axis=1, inplace=True)

    # Merge ELO ratings from ddf and rename the ELO to previous_elo
    current_race = current_race.merge(
        ddf[['driverId', 'ELO', 'forename', 'surname']],
        on='driverId',
        how='left'
    ).rename(columns={'ELO': 'previous_elo'})
    
    current_race['updated_elo'] = current_race['previous_elo']

    for index, row in current_race.iterrows():
        if pd.isna(row['percent_difference']):
            # If percent_difference is null, skip calculations
            continue
        else:
            # Proceed with ELO calculations only if percent_difference is not null
            elo_change = 0
            for _, opp_row in current_race.iterrows():
                # Ensure opponent also does not have a null percent_difference
                if row['driverId'] != opp_row['driverId'] and not pd.isna(opp_row['percent_difference']):
                    elo_change += calculate_elo_change(row, opp_row)
            current_race.at[index, 'updated_elo'] += elo_change

    # Update ddf with the new ELO scores
    for index, row in current_race.iterrows():
        ddf.loc[ddf['driverId'] == row['driverId'], 'ELO'] = row['updated_elo']
        
    for index, row in current_race.iterrows():
        # Find the current maximum ELO for the driver and update if the new value is higher
        mask = ddf['driverId'] == row['driverId']
        current_max_elo = ddf.loc[mask, 'MAX ELO']
        new_elo = row['updated_elo']
        # Update only if the new ELO is greater than the current max ELO
        ddf.loc[mask & (new_elo > current_max_elo), 'MAX ELO'] = new_elo

    updates = current_race[['raceId', 'driverId', 'previous_elo', 'updated_elo']]

    # Merge the updates back into the main DataFrame
    df = df.merge(updates, on=['raceId', 'driverId'], how='left', suffixes=('', '_update'))

    # Update the main DataFrame columns
    df['previous_elo'] = df['previous_elo_update'].fillna(df['previous_elo'])
    df['updated_elo'] = df['updated_elo_update'].fillna(df['updated_elo'])

    # Drop the temporary update columns
    df.drop(['previous_elo_update', 'updated_elo_update'], axis=1, inplace=True)
    #break
    

In [ ]:
current_race

In [ ]:
ddf

In [ ]:
df = df.merge(
    ddf[['driverId', 'forename', 'surname']],
    on='driverId',
    how='left'
)
df

In [ ]:
ddf.to_csv('ddf.csv')

In [ ]:
df.to_csv('finalELO.csv')